#### Stat Distribution

In [20]:
import pandas as pd
import os 
import numpy as np

os.chdir('C:/Users/dalto/OneDrive/Pictures/Documents/Projects/Coding Projects/woba modeling/data/')

In [21]:
woba = pd.read_csv('pitch/wobacon.csv')
full = pd.read_csv('sim_results/full_wobacon_23_25.csv')

In [22]:
woba = woba[woba['pa'] > 150]
print(woba.groupby('year')['wobacon'].describe())

      count      mean       std    min      25%     50%     75%    max
year                                                                  
2015  389.0  0.365769  0.057078  0.241  0.32800  0.3670  0.3990  0.583
2016  393.0  0.371476  0.055417  0.236  0.33000  0.3690  0.4090  0.547
2017  390.0  0.376797  0.057210  0.237  0.33600  0.3690  0.4140  0.600
2018  392.0  0.370952  0.058383  0.228  0.32975  0.3685  0.4060  0.596
2019  411.0  0.381406  0.059488  0.222  0.33750  0.3800  0.4190  0.644
2020  201.0  0.393502  0.062043  0.246  0.34800  0.3900  0.4320  0.548
2021  409.0  0.370707  0.058490  0.232  0.33400  0.3610  0.4080  0.567
2022  417.0  0.359878  0.058429  0.203  0.32200  0.3570  0.3980  0.602
2023  404.0  0.372688  0.059625  0.192  0.32900  0.3690  0.4110  0.557
2024  410.0  0.361239  0.057412  0.217  0.32325  0.3550  0.3970  0.617
2025  300.0  0.370020  0.061904  0.221  0.32500  0.3640  0.4075  0.651


In [23]:
target_p25 = .325
target_p75 = .400


original_p25 = full['monte'].quantile(0.25)
original_p75 = full['monte'].quantile(0.75)

slope = (target_p75 - target_p25) / (original_p75 - original_p25)
intercept = target_p25 - slope * original_p25

full['monte'] = slope * full['monte'] + intercept

In [24]:
full.to_csv('transforms/wobacon.csv', index=False)